In [591]:
from pandas import Series, DataFrame
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


### Q1 - Read in data

In [592]:
gold_df = pd.read_csv("gold.txt", sep='\t', names=['url','category'])
labels_df = pd.read_csv("labels.txt", sep="\t", names=['turk','url','category'])
#gold_df = gold_df.drop_duplicates('url')
#labels_df = labels_df.drop_duplicates(['turk','url'])
gold_df[:5]

,url,category
0,http://0800-horoscope.com,G
1,http://18games.net,X
2,http://1pixelout.net,G
3,http://1porno.com/004/teens/index.html,X
4,http://1stpussy.com/005/16/index.html,X


In [593]:
labels_df[labels_df.duplicated(subset=['turk','url'])]

,turk,url,category
2,A1PXXEOGQ76RNJ,http://000.cc,G
23,A1PXXEOGQ76RNJ,http://000relationships.com,G
36,A4GP97LAQCCHJ,http://000relationships.com,G
92,A1PXXEOGQ76RNJ,http://01house.cn,G
94,A21US576U8SCO4,http://01house.cn,G
...,...,...,...
92627,A1PXXEOGQ76RNJ,http://zucaiba.com,G
92636,A1S0C0EB7A6FFM,http://zumin-ads.com,G
92644,APTFS1S88K5VC,http://zumin-ads.com,G
92657,A1PXXEOGQ76RNJ,http://zvezdanews.ru,G


In [594]:
labels_df[labels_df['url'] == 'http://000.cc']

,turk,url,category
0,A1OT3A29R9N1DG,http://000.cc,P
1,A1PXXEOGQ76RNJ,http://000.cc,G
2,A1PXXEOGQ76RNJ,http://000.cc,G
3,A21US576U8SCO4,http://000.cc,G
4,A2LGX47NN7C5D3,http://000.cc,G
5,A2OVKTB7VNY8EW,http://000.cc,G
6,A31Z3E5SLATLML,http://000.cc,P
7,AAVCPZ8WTCFVK,http://000.cc,G
8,AHY98D5P05XIV,http://000.cc,G
9,APTFS1S88K5VC,http://000.cc,G


In [595]:
labels_df[:5]

,turk,url,category
0,A1OT3A29R9N1DG,http://000.cc,P
1,A1PXXEOGQ76RNJ,http://000.cc,G
2,A1PXXEOGQ76RNJ,http://000.cc,G
3,A21US576U8SCO4,http://000.cc,G
4,A2LGX47NN7C5D3,http://000.cc,G


### Q2 - Split into two DataFrames

In [596]:
gold_urls = gold_df['url']
labels_on_gold = labels_df[labels_df['url'].isin(gold_urls)]
labels_unknown = labels_df[~labels_df['url'].isin(gold_urls)]
print(len(labels_on_gold), " ", len(labels_unknown))
labels_on_gold[:5]

3324   89397


,turk,url,category
245,A1253FXHCZ9CWM,http://0800-horoscope.com,G
246,A153PKAL7OAY36,http://0800-horoscope.com,G
247,A1FV9SAPL5C6KY,http://0800-horoscope.com,G
248,A1JTOT0DWM6QGL,http://0800-horoscope.com,G
249,A1PXXEOGQ76RNJ,http://0800-horoscope.com,G


### Q3 - Compute accuracies of turks
indexed by turk, and has two columns: the number of ratings, and the average correctness of ratings for each turk (both on gold set urls).

In [597]:
turks = labels_on_gold['turk'].unique()
gold_label = labels_on_gold.merge(gold_df,left_on='url',right_on='url',suffixes=('_turk','_true'))
gold_label[:2]

,turk,url,category_turk,category_true
0,A1253FXHCZ9CWM,http://0800-horoscope.com,G,G
1,A153PKAL7OAY36,http://0800-horoscope.com,G,G


In [598]:
def get_numRated(x):
    return len(gold_label[gold_label['turk']==x]['url'])

def get_correct(x):
    cur = gold_label[gold_label['turk']==x]
    correct = len(cur[cur['category_true']==cur['category_turk']])
    return correct
    
grp = gold_label.groupby('turk')
gold_label['numRated'] = gold_label['turk'].map(get_numRated)
gold_label['correct'] = gold_label['turk'].map(get_correct)
gold_label['correctness'] = gold_label['correct']/gold_label['numRated']

In [616]:
rater_goodness = gold_label[['turk','numRated','correctness']].drop_duplicates()
rater_goodness = rater_goodness.set_index('turk').fillna(0)
rater_goodness[:3]
len(rater_goodness)

269

### Q4 - Odds ratios

In [600]:
def get_odd(x):
    return x/(1.001-x)

rater_goodness['odds'] = rater_goodness['correctness'].map(get_odd)
#CANNOT DROP DUPLICATES cuz turk is index, not included
len(rater_goodness)

269

### Q5 - Most accurate turks
List the top 10 most accurate turks who have rated at least 20 gold set URLs.

In [617]:
turk_correctness = gold_label[['turk','numRated','correctness']].drop_duplicates()
rated_20more = turk_correctness[turk_correctness['numRated'] >= 20].sort_values(by='correctness', ascending=False)[:10]
rated_20more

,turk,numRated,correctness
205,A2U0R4X38GUKZE,20,0.950000
6,A22C0PJUBFJTI0,36,0.916667
82,A23YQUBXZPKILZ,24,0.875000
48,ATVALOQVDCMZW,103,0.854369
42,A1HIXWH4OXT8S4,40,0.825000
32,A3220HG1O83HQ4,22,0.818182
43,A20PWAB7G3HDHU,20,0.800000
125,A32W20KGQXS0LL,25,0.800000
86,AJSJVK40F5HM6,28,0.785714
11,A31OCN4MNHUQ6W,184,0.777174


   ### Q6 - Rating counts versus accuracy

In [602]:
import statsmodels.api as sm
from patsy import dmatrices

In [603]:
y, X = dmatrices('correctness ~ numRated', data=gold_label, return_type='dataframe')
model = sm.OLS(y, X)       # Set up the model
result = model.fit()       # Fit model (find the intercept and slopes)
print(result.summary())
print("p-values: \n", result.pvalues)

                            OLS Regression Results                            
Dep. Variable:            correctness   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     21.37
Date:                Thu, 06 Aug 2020   Prob (F-statistic):           3.93e-06
Time:                        12:25:21   Log-Likelihood:                 1128.1
No. Observations:                3324   AIC:                            -2252.
Df Residuals:                    3322   BIC:                            -2240.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.6649      0.005    144.174      0.0

 - As shown above, we can see that the p values are small enough to justify the positive relationship between correctedness and the number of ratings. However, the r square is very small, so it is a poor fit that only accounts for a small part of the data variation.

### Q7 - Overall predicted odds
all (URL, category) pairs：don't need to include URLs which were never rated by a "reliable" turker (i.e., a turker who has rated more gold URLs than 75% of turkers, etc.) The same idea holds for Q8 and Q9 too.

 - for each url u that is not in the gold set "labels_unknown"
<br>
and each category c. For the pair (u, c), calculate the product of odds of all turks who 
<br>
(a) rated url u as category c
<br>
(b) get turks have rated more gold set urls than 75% of all turks who rated at least one gold-set url

In [604]:
cut = turk_correctness['numRated'].quantile(q = 0.75)
top_25 = turk_correctness[turk_correctness['numRated'] > cut]
top_25.sort_values(by='numRated',ascending=False)[-3:]

,turk,numRated,correctness
1593,AC0YIZF6G5V9G,13,0.769231
202,A1IB9WML70CU89,12,0.833333
16,A4GP97LAQCCHJ,12,0.583333


In [605]:
#df of unkown urls rated by reliable turks
unknown_re = labels_unknown[labels_unknown['turk'].isin(top_25['turk'])]
print(len(labels_unknown), len(unknown_re))

89397 40621


In [606]:
categories = unknown_re['category'].unique()
test_url = unknown_re.loc[63]['url']# for testing purposes

In [607]:
unknown_re[unknown_re['url']==test_url]#http://000relationships.com，

,turk,url,category
63,A1253FXHCZ9CWM,http://007absolutehardcore.com/early_cumshots/...,P
64,A153PKAL7OAY36,http://007absolutehardcore.com/early_cumshots/...,X
65,A1JTOT0DWM6QGL,http://007absolutehardcore.com/early_cumshots/...,P
66,A22C0PJUBFJTI0,http://007absolutehardcore.com/early_cumshots/...,X
67,A270HOAHEVIFW0,http://007absolutehardcore.com/early_cumshots/...,R
69,A31OCN4MNHUQ6W,http://007absolutehardcore.com/early_cumshots/...,X
70,A35GER5TWMH9VP,http://007absolutehardcore.com/early_cumshots/...,X
71,A3GE97GX3OXJTW,http://007absolutehardcore.com/early_cumshots/...,R
72,AZ1S5BIG5R6P6,http://007absolutehardcore.com/early_cumshots/...,R


In [608]:
def loc_odd(x):
    return rater_goodness.loc[x,'odds']

unknown_re['odd'] = unknown_re['turk'].map(loc_odd)
unknown_re[unknown_re['url']==test_url]

/Users/clairem/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,turk,url,category,odd
63,A1253FXHCZ9CWM,http://007absolutehardcore.com/early_cumshots/...,P,1.069214
64,A153PKAL7OAY36,http://007absolutehardcore.com/early_cumshots/...,X,2.600369
65,A1JTOT0DWM6QGL,http://007absolutehardcore.com/early_cumshots/...,P,3.129890
66,A22C0PJUBFJTI0,http://007absolutehardcore.com/early_cumshots/...,X,10.869565
67,A270HOAHEVIFW0,http://007absolutehardcore.com/early_cumshots/...,R,2.562278
69,A31OCN4MNHUQ6W,http://007absolutehardcore.com/early_cumshots/...,X,3.472222
70,A35GER5TWMH9VP,http://007absolutehardcore.com/early_cumshots/...,X,2.988048
71,A3GE97GX3OXJTW,http://007absolutehardcore.com/early_cumshots/...,R,1.925391
72,AZ1S5BIG5R6P6,http://007absolutehardcore.com/early_cumshots/...,R,2.491281


In [614]:
tb = pd.pivot_table(unknown_re, index='url',columns='category',values=['odd'],aggfunc=np.prod)
tb = tb.fillna(0)
tb

odd                         
category                             G         P    R         X
url                                                            
http://0-101.net              2.155963  0.000000  0.0  0.000000
http://000.cc                 1.460583  0.000000  0.0  0.000000
http://0000.jp               14.488244  0.000000  0.0  0.000000
http://000relationships.com   5.681060  1.851852  0.0  0.000000
http://000vitamins.com        3.784982  0.000000  0.0  0.000000
...                                ...       ...  ...       ...
http://zwinky.com            16.141443  0.000000  0.0  0.000000
http://zylom.com              1.735526  0.000000  0.0  0.000000
http://zynga.com              3.704611  0.000000  0.0  0.000000
http://zz868.com              1.047852  1.851852  0.0  0.000000
http://zzx.cc                 1.330229  0.000000  0.0  1.620746

[10610 rows x 4 columns]

### Q8 - Predicted categories
top category: category with the highest overall odds for that url<br>
top odds: overall odds for that top category

In [615]:
def topOdds(x):
    return tb.loc[x].max()

def topCat(x):
    return tb.loc[x].idxmax()[1]

urls= tb.index.to_series()
tb['top odds'] = urls.map(topOdds)
tb['top category'] = urls.map(topCat)
tb

odd                            top odds  \
category                             G         P    R         X              
url                                                                          
http://0-101.net              2.155963  0.000000  0.0  0.000000   2.155963   
http://000.cc                 1.460583  0.000000  0.0  0.000000   1.460583   
http://0000.jp               14.488244  0.000000  0.0  0.000000  14.488244   
http://000relationships.com   5.681060  1.851852  0.0  0.000000   5.681060   
http://000vitamins.com        3.784982  0.000000  0.0  0.000000   3.784982   
...                                ...       ...  ...       ...        ...   
http://zwinky.com            16.141443  0.000000  0.0  0.000000  16.141443   
http://zylom.com              1.735526  0.000000  0.0  0.000000   1.735526   
http://zynga.com              3.704611  0.000000  0.0  0.000000   3.704611   
http://zz868.com              1.047852  1.851852  0.0  0.000000   1.851852   
http://zzx.cc                 1.330229  0.000000  0.0  1.620746   1.620746   

                            top category  
category                                  
url                                       
http://0-101.net                       G  
http://000.cc                          G  
http://0000.jp                         G  
http://000relationships.com            G  
http://000vitamins.com                 G  
...                                  ...  
http://zwinky.com                      G  
http://zylom.com                       G  
http://zynga.com                       G  
http://zz868.com                       P  
http://zzx.cc                          X  

[10610 rows x 6 columns]

In [582]:
result_75 = DataFrame({'top category':tb['top category'], 'top odds':tb['top odds']},index=tb.index)
result_75

,top category,top odds
url,,
http://0-101.net,G,2.155963
http://000.cc,G,1.460583
http://0000.jp,G,14.488244
http://000relationships.com,G,5.681060
http://000vitamins.com,G,3.784982
...,...,...
http://zwinky.com,G,16.141443
http://zylom.com,G,1.735526
http://zynga.com,G,3.704611


### Q9 - Predicted categories using more turks

In [583]:
cut2 = turk_correctness['numRated'].quantile(q = 0.25)
top_75 = turk_correctness[turk_correctness['numRated'] > cut2]
unknown_75 = labels_unknown[labels_unknown['turk'].isin(top_75['turk'])]
unknown_75['odds'] = unknown_75['turk'].map(loc_odd)
unknown_75
tb_75 = pd.pivot_table(unknown_75, index='url',columns='category',values=['odds'],aggfunc=np.prod)
tb_75 = tb_75.fillna(0)
print(len(labels_unknown), len(unknown_75))
tb_75

89397 66210


/Users/clairem/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


odds                                 
category                                G            P         R         X
url                                                                       
http://0-101.net             2.155963e+00     0.000000  0.000000  0.000000
http://000.cc                2.181050e+03     0.998004  0.000000  0.000000
http://0000.jp               2.877556e+07     0.000000  0.000000  0.000000
http://000relationships.com  0.000000e+00     1.851852  0.332889  0.000000
http://000vitamins.com       5.014149e+00     0.000000  0.000000  0.000000
...                                   ...          ...       ...       ...
http://zwinky.com            2.125151e+02  1000.000000  0.000000  0.000000
http://zylom.com             2.873345e+03     0.000000  0.000000  0.000000
http://zynga.com             5.442125e+01     0.000000  0.000000  0.000000
http://zz868.com             1.209566e+13     1.851852  0.000000  0.000000
http://zzx.cc                1.330229e+00     0.998004  0.000000  1.620746

[10700 rows x 4 columns]

In [584]:
def topOdds75(x):
    return tb_75.loc[x].max()

def topCat75(x):
    return tb_75.loc[x].idxmax()[1]

tb_75['top odds'] = urls.map(topOdds75)
tb_75['top category'] = urls.map(topCat75)
tb_75

odds                                   \
category                                G            P         R         X   
url                                                                          
http://0-101.net             2.155963e+00     0.000000  0.000000  0.000000   
http://000.cc                2.181050e+03     0.998004  0.000000  0.000000   
http://0000.jp               2.877556e+07     0.000000  0.000000  0.000000   
http://000relationships.com  0.000000e+00     1.851852  0.332889  0.000000   
http://000vitamins.com       5.014149e+00     0.000000  0.000000  0.000000   
...                                   ...          ...       ...       ...   
http://zwinky.com            2.125151e+02  1000.000000  0.000000  0.000000   
http://zylom.com             2.873345e+03     0.000000  0.000000  0.000000   
http://zynga.com             5.442125e+01     0.000000  0.000000  0.000000   
http://zz868.com             1.209566e+13     1.851852  0.000000  0.000000   
http://zzx.cc                1.330229e+00     0.998004  0.000000  1.620746   

                                 top odds top category  
category                                                
url                                                     
http://0-101.net             2.155963e+00            G  
http://000.cc                2.181050e+03            G  
http://0000.jp               2.877556e+07            G  
http://000relationships.com  1.851852e+00            P  
http://000vitamins.com       5.014149e+00            G  
...                                   ...          ...  
http://zwinky.com            1.000000e+03            P  
http://zylom.com             2.873345e+03            G  
http://zynga.com             5.442125e+01            G  
http://zz868.com             1.209566e+13            G  
http://zzx.cc                1.620746e+00            X  

[10700 rows x 6 columns]

In [585]:
result_25 = DataFrame({'top category':tb_75['top category'], 'top odds':tb_75['top odds']},index=tb_75.index)
results = result_25.join(result_75, on=None,how='left',lsuffix='_rst25', rsuffix='_rst75', sort=False)
results

,top category_rst25,top odds_rst25,top category_rst75,top odds_rst75
url,,,,
http://0-101.net,G,2.155963e+00,G,2.155963
http://000.cc,G,2.181050e+03,G,1.460583
http://0000.jp,G,2.877556e+07,G,14.488244
http://000relationships.com,P,1.851852e+00,G,5.681060
http://000vitamins.com,G,5.014149e+00,G,3.784982
...,...,...,...,...
http://zwinky.com,P,1.000000e+03,G,16.141443
http://zylom.com,G,2.873345e+03,G,1.735526
http://zynga.com,G,5.442125e+01,G,3.704611


In [586]:
top_cats = results[['top category_rst75','top category_rst25']]
top_cats['urls'] = top_cats.index.to_series()

/Users/clairem/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [587]:
piv = pd.pivot_table(top_cats, index='top category_rst75', columns='top category_rst25', values='urls', aggfunc='count')
piv

top category_rst25,G,P,R,X
top category_rst75,,,,
G,8327,574,186,216
P,189,328,47,19
R,21,34,128,25
X,27,6,26,457


 - The most errors are where result_75 perdicted 'G' but result_25 perdicted 'P'